In [ ]:
# get dataset
! kaggle datasets download -f actors.csv gsimonx37/letterboxd
! unzip actors.csv.zip
! rm actors.csv.zip

# get whole repo if running in google colab
! git clone https://github.com/mattia01017/movie-actor-mb-analysis
! pip install -r movie-actor-mb-analysis/requirements.txt

# setup Spark
import os
import findspark
! apt-get install openjdk-8-jdk-headless -qq > /dev/null
! wget -q http://archive.apache.org/dist/spark/spark-3.1.1/spark-3.1.1-bin-hadoop3.2.tgz
! tar xf spark-3.1.1-bin-hadoop3.2.tgz
! pip install -q findspark
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.1-bin-hadoop3.2"
findspark.init("spark-3.1.1-bin-hadoop3.2")

In [ ]:
import json
import numpy as np
from pympler.asizeof import asizeof
from typing import Iterable
from collections import defaultdict, Counter
from itertools import combinations, count
from functools import reduce
from pyspark.sql import SparkSession
from pyspark import RDD
import gc

CHUNKS = 5

spark = SparkSession.builder\
    .appName("movie-actor-mb-analysis")\
    .config("spark.default.parallelism", str(CHUNKS))\
    .getOrCreate()

# Market-basket analysis of Letterboxd dataset

## Preprocessing

Starting from a csv table that associate film identifiers to actors, we generate the list of baskets and save it to disk. The list is sorted by movie ID to obtain a predictable order and is not strictly needed.

In [ ]:
df = spark.read.csv("actors.csv", header=True, sep=",", mode="DROPMALFORMED")
df.rdd\
    .map(lambda x: (x["id"], x["name"]))\
    .groupByKey()\
    .sortByKey()\
    .map(lambda x: json.dumps(list(x[1])))\
    .saveAsTextFile("baskets")

We define an iterator that implement a lazy loading of baskets from files. In this way, we can load in memory a basket at a time instead of the whole dataset.

In [2]:
class Baskets:
    def __init__(self, parts_path) -> None:
        self.parts_path = parts_path
        
    def _file_name(self):
        return "{0}/part-{1:0>5}".format(self.parts_path, self.part)
        
    def __iter__(self):
        self.part = 0
        self.file = open(self._file_name())
        return self
    
    def __next__(self):
        try:
            line = next(self.file)
        except StopIteration: 
            self.file.close()
            self.part += 1
            try:
                self.file = open(self._file_name())
                return self.__next__()
            except FileNotFoundError:
                raise StopIteration
        return tuple(json.loads(line))
    

## Algorithms

For the analysis, the Savasere, Omiecinski and Navathe (SON) algorithm will be implemented using the Park, Cheng and Yu (PCY) algorithm for retrieving frequent itemsets in the chunks.

### PCY

First, a very simple class representing a bitmap useful for the PCY algorithm implementation is defined.

In [2]:
class Bitmap:
    def __init__(self, bits_arr: list) -> None:
        self.bytes = np.packbits(bits_arr, bitorder="little")

    def get(self, index: int) -> bool:
        return bool(self.bytes[index // 8] & pow(2, index % 8))

    def set(self, index: int):
        self.bytes[index // 8] |= pow(2, index % 8)

    def __repr__(self) -> str:
        return " ".join(["{0:08b}".format(b) for b in self.bytes])

After that, the PCY algorithm can be implemented. The garbage collector is manually triggered for deleting from memory the counters immediately after the bitmap creation.

In [3]:
def hash_t(itemset: tuple) -> int:
    """hash tuple ignoring order"""
    return reduce(lambda p, c: p ^ hash(c), itemset, 0)


def apriori(
    baskets: Iterable[tuple[str]],
    threshold: int,
    freq_items: list[str],
    freq_couples: list[tuple],
    iset_len_limit: int | None = None
) -> list[tuple]:
    freq_itemsets = freq_couples
    sizes_iter = iset_len_limit and range(3, iset_len_limit+1) or count(3)
    for s in sizes_iter:
        counters = defaultdict(int)
        for basket in baskets:
            filtered_basket = (item for item in basket if item in freq_items)
            filtered_basket = set(sum(
                (list(itemset) for itemset in combinations(filtered_basket, s-1) if tuple(sorted(itemset)) in freq_itemsets), []
            ))
            for itemset in combinations(filtered_basket, s):
                counters[tuple(sorted(itemset))] += 1
        new_frequent = [itemset for itemset, count in counters.items() if count > threshold]
        if len(new_frequent) == 0: break
        freq_itemsets.extend(new_frequent)
    
    return freq_itemsets


def PCY(
    baskets: Iterable[tuple[str]],
    threshold: int,
    buckets: int,
    iset_len_limit: int | None = None
) -> list[tuple]:  
    item_counts = Counter()
    itemset_counts = np.zeros(buckets, dtype=np.uint32)

    for basket in baskets:
        for item in basket:
            item_counts[item] += 1
        for itemset in combinations(basket, 2):
            itemset_counts[hash_t(itemset) % buckets] += 1

    freq_items = [item for item, count in item_counts.items() if count > threshold]
    del item_counts
    gc.collect()

    bitmap = Bitmap([count > threshold for count in itemset_counts])
    del itemset_counts
    gc.collect()
    
    freq_couples = [
        tuple(sorted(itemset))
        for itemset in combinations(freq_items, 2)
        if bitmap.get(hash_t(itemset) % buckets)
    ]
    
    return apriori(baskets, threshold, freq_items, freq_couples, iset_len_limit)

In this implementation, the itemsets with cardinality larger than 2 are obtained using the apriori algorithm, as a low number of those itemset is expected to be frequent.

PCY alone can be used to retrieve frequent itemset (buckets) using a single node for computation. To avoid long execution times, in this section only frequent couples will be computed. Unsetting the `iset_len_limit` optional parameter in the cell below (or setting it to an higher value) will force the search to look for frequent itemsets with higher cardinality. The next cell should take around 5 minutes in a Google Colab CPU runtime.  

In [ ]:
PCY(Baskets("baskets"), 170, int(1e6), iset_len_limit=2)

The result contains also false positive couples, that is infrequent couples put in a frequent buckets. To remove false positives, another pass would be needed. The problem is best delved during SON implementation.

### SON

Execution times can be improved by using SON, parallelizing the execution of PCY on a number of chunks and combining the results. The Apache Spark framework is used for the implementation of the SON algorithm. 

We start by loading the basket files:

In [4]:
df = spark.read.text("baskets")
baskets: RDD = df.rdd.map(lambda row: tuple(json.loads(row.value)))

In [5]:
baskets.count()

634300

### Memory usage

The objective is to have the maximum memory usage without swapping and thus thrashing. The main elements to store in memory are:
- The hash table of item counters
- The array of bucket counters
- The bitmap of frequent buckets

The memory usage of the bitmap and the array of counters is easy to predict given the size, more tricky is doing it for the hash table of counters. For this purpose, we use a tool for observing memory behaviour of Python objects, namely Pympler. The `asizeof` method return an approximation of the memory usage of an object.

We measure the size of the `Counter` object after counting all items.

In [6]:
counter = Counter(baskets.flatMap(lambda x: x).collect())
print("{0:.3f} MB".format(asizeof(counter) / 1e6))

157.977 MB


Thus, we can assume that a single node won't use more than 200 MB for storing the item counters. The remaining space can be used to store the bucket counters. Assuming we want to use up to 2 GB of memory for each computing node, we can use a number of buckets with 32-bit unsigned integer counters equal to:
$$
\frac{2 \cdot 10^9 \text{ B} - 2 \cdot 10^8 \text{ B}}{4 \text{ B}} = 4.5 \cdot 10^8 \text{ buckets}
$$

while the bitmap will occupy $1/32$ of the space, that is $56 \text{ MB}$.

The last parameters to tune are the thresholds for labelling an itemset as frequent. This will be chosen experimentally in a way to obtain an output of reasonable size, say around 50 itemsets in the whole basket list. 

### Map-reduce implementation

To avoid long times in resource constrained environments, only a randomly sampled list of baskets is used during the analysis. For the same reason, the used number of bucket will be smaller than the one computed above.

The result of the analysis on the original dataset using $10^7$ buckets and a threshold of 100 is reported in the `frequent_itemset.json` file in the repository. The next cell shows an execution on a much smaller set of baskets, considering only couples. To search from larger baskets and to retrieve itemsets with larger cardinality, simply increase respectively the `SAMPLE_FRACTION` and `ISET_LEN_LIMIT` constants, eventually setting to `None` the latter to remove the cardinality limit.

The cell below takes around 8 minutes to generate a result in a Google Colab CPU runtime.

In [ ]:
BUCKETS = int(1e7)
THRESHOLD = 45
ISET_LEN_LIMIT = 2
SAMPLE_FRACTION = .25

if SAMPLE_FRACTION < 1:
    baskets = baskets.sample(False, SAMPLE_FRACTION, 2)

def count_occurrences(
    baskets: Iterable, 
    candidates: Iterable, 
    iset_len_limit: int | None = None
):
    out = {tuple(sorted(k)):0 for k in candidates}
    for basket in baskets:
        sizes_iter = iset_len_limit and range(2, iset_len_limit+1) or count(2)
        for size in sizes_iter:
            exit = True
            for itemset in combinations(basket, size):
                itemset = tuple(sorted(itemset))
                if itemset in out:
                    exit = False
                    out[itemset] += 1
            if exit: break        
    return out.items()

def SON(baskets: RDD, threshold: int, chunks: int, buckets: int) -> list[tuple]:
    candidates = baskets\
        .mapPartitions(lambda chunk: PCY(list(chunk), threshold // chunks, buckets, ISET_LEN_LIMIT))\
        .distinct()\
        .collect()
    frequent_itemsets = baskets\
        .mapPartitions(lambda chunk: count_occurrences(chunk, candidates, ISET_LEN_LIMIT))\
        .reduceByKey(lambda a, b: a + b)\
        .filter(lambda x:  x[1] > THRESHOLD)\
        .collect()
    frequent_itemsets.sort(key=lambda x: -x[1])
    return frequent_itemsets

frequent_itemsets = SON(baskets, THRESHOLD, CHUNKS, BUCKETS)

for x in frequent_itemsets[:10]: print(x)
with open("frequent_itemsets.json", "w") as f:
    json.dump([{"set": s[0], "count": s[1]} for s in frequent_itemsets], f, indent=2)

print("Number of frequent itemsets:", len(frequent_itemsets))
print("Whole list of frequent itemsets saved in 'frequent_itemsets.json'")
